In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.multiclass import check_classification_targets

class AdaBoostClassifier(BaseEstimator, ClassifierMixin):
    """
    Реализация алгоритма AdaBoost для бинарной и мультиклассовой классификации
    """
    
    def __init__(self, n_estimators=50, max_depth=1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.classifier_weights = []
        self.classifiers = []
        self.weights_history = []
        self.label_encoder = LabelEncoder()
    
    def fit(self, X, y):
        """
        Обучение AdaBoost модели с обработкой различных типов меток
        """
        # Проверка и преобразование меток
        check_classification_targets(y)
        y_encoded = self.label_encoder.fit_transform(y)
        
        # Преобразуем метки к формату {-1, 1} для бинарной классификации
        if len(self.label_encoder.classes_) == 2:
            y_transformed = np.where(y_encoded == 0, -1, 1)
        else:
            # Для мультиклассовой используем оригинальные метки
            y_transformed = y_encoded
        
        n_samples = X.shape[0]
        
        # 1. ИНИЦИАЛИЗАЦИЯ ВЕСОВ ОБЪЕКТОВ
        weights = np.ones(n_samples) / n_samples
        self.weights_history.append(weights.copy())
        
        for t in range(self.n_estimators):
            # 2. ОБУЧЕНИЕ БАЗОВОГО КЛАССИФИКАТОРА
            classifier = self._train_base_classifier(X, y_transformed, weights)
            
            # 3. ВЫЧИСЛЕНИЕ ОШИБКИ КЛАССИФИКАТОРА
            error = self._calculate_classifier_error(X, y_transformed, weights, classifier)
            
            # 4. ВЫЧИСЛЕНИЕ ВЕСА КЛАССИФИКАТОРА
            alpha = self._calculate_alpha(error)
            self.classifier_weights.append(alpha)
            self.classifiers.append(classifier)
            
            # 5. ОБНОВЛЕНИЕ ВЕСОВ ОБЪЕКТОВ
            weights = self._update_sample_weights(X, y_transformed, weights, classifier, alpha)
            self.weights_history.append(weights.copy())
            
            # Ранняя остановка если ошибка равна 0 или слишком мала
            if error < 1e-10:
                break
    
    def _train_base_classifier(self, X, y, weights):
        """Обучение слабого классификатора"""
        classifier = DecisionTreeClassifier(max_depth=self.max_depth)
        classifier.fit(X, y, sample_weight=weights)
        return classifier
    
    def _calculate_classifier_error(self, X, y, weights, classifier):
        """Вычисление взвешенной ошибки"""
        predictions = classifier.predict(X)
        
        # Убеждаемся, что типы данных совместимы
        predictions = predictions.astype(y.dtype)
        incorrect = (predictions != y)
        
        error = np.sum(weights[incorrect]) / np.sum(weights)
        return error
    
    def _calculate_alpha(self, error):
        """Вычисление веса классификатора"""
        epsilon = 1e-10
        error = np.clip(error, epsilon, 1 - epsilon)
        return 0.5 * np.log((1 - error) / error)
    
    def _update_sample_weights(self, X, y, weights, classifier, alpha):
        """Обновление весов объектов"""
        predictions = classifier.predict(X)
        
        # Убеждаемся, что типы данных совместимы
        predictions = predictions.astype(y.dtype)
        
        # Правильно вычисляем множители
        incorrect_mask = (predictions != y)
        multipliers = np.exp(alpha * incorrect_mask.astype(float))
        
        # Обновляем веса
        new_weights = weights * multipliers
        new_weights /= np.sum(new_weights)  # Нормализация
        
        return new_weights
    
    def predict(self, X):
        """Предсказание меток для новых данных"""
        # Взвешенное голосование ансамбля
        predictions = np.zeros(X.shape[0])
        
        for alpha, classifier in zip(self.classifier_weights, self.classifiers):
            pred = classifier.predict(X)
            predictions += alpha * pred
        
        # Для бинарной классификации возвращаем к оригинальным меткам
        if len(self.label_encoder.classes_) == 2:
            final_predictions = np.where(predictions >= 0, 
                                       self.label_encoder.classes_[1], 
                                       self.label_encoder.classes_[0])
        else:
            # Для мультиклассовой - используем знак или округление
            final_predictions = self.label_encoder.inverse_transform(
                np.round(predictions).astype(int)
            )
        
        return final_predictions
    
    def predict_proba(self, X):
        """Предсказание вероятностей классов"""
        score = np.zeros(X.shape[0])
        
        for alpha, classifier in zip(self.classifier_weights, self.classifiers):
            pred = classifier.predict(X)
            score += alpha * pred
        
        # Преобразуем score в вероятности
        if len(self.label_encoder.classes_) == 2:
            probabilities = 1 / (1 + np.exp(-2 * score))
            return np.column_stack([1 - probabilities, probabilities])
        else:
            # Для мультиклассовой используем softmax
            from scipy.special import softmax
            return softmax(score.reshape(-1, 1), axis=1)

# ДЕМОНСТРАЦИЯ С ПРАВИЛЬНОЙ ОБРАБОТКОЙ ДАННЫХ
def demonstrate_adaboost_fixed():
    """Демонстрация работы AdaBoost с правильной обработкой данных"""
    from sklearn.datasets import make_classification
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    
    # Генерация синтетических данных
    X, y = make_classification(n_samples=1000, n_features=20, 
                              n_informative=15, n_redundant=5,
                              random_state=42)
    
    # Разделение на train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Создание и обучение AdaBoost
    adaboost = AdaBoostClassifier(n_estimators=100, max_depth=1)
    adaboost.fit(X_train, y_train)
    
    # Предсказание и оценка качества
    y_pred = adaboost.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Точность AdaBoost: {accuracy:.4f}")
    print(f"Количество классификаторов: {len(adaboost.classifiers)}")
    print(f"Веса классификаторов: {adaboost.classifier_weights[:5]}...")

# Запуск демонстрации
if __name__ == "__main__":
    demonstrate_adaboost_fixed()

Точность AdaBoost: 0.8100
Количество классификаторов: 100
Веса классификаторов: [np.float64(0.4118000344786902), np.float64(0.21801223556400137), np.float64(0.2582862763391271), np.float64(0.23846063311561497), np.float64(0.21380702536287824)]...
